In [90]:
import pandas as pd
import folium
from geopy.geocoders import ArcGIS
import random 
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from sklearn.datasets import make_blobs
from yellowbrick.cluster import KElbowVisualizer
%matplotlib inline

In [ ]:
colors = ['blue', 'green', 'purple', 'orange','lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
len(colors)

In [ ]:
ad =  pd.read_excel('book3.xlsx');
# ad.drop(ad.columns[[0,1,2,3]], axis=1, inplace=True)
ad.head(5)

In [93]:
ad['coordinates']=ad['Địa chỉ'].apply(ArcGIS().geocode) 
# ad['Địa chỉ']

In [94]:
ad['Latitude'] = ad['coordinates'].apply(lambda x: x.latitude)
ad['Logtitude'] = ad['coordinates'].apply(lambda x: x.longitude)

# Phân cụm các điểm 


In [ ]:
print("Nhập số cụm cần lấy: ")
k = int(input())


# tạo một bảng khác chứa cluter và lat log

In [96]:
coord = pd.DataFrame()

In [97]:
km = KMeans(n_clusters = k)
y_predicted = km.fit_predict(ad[['Latitude','Logtitude']])
ad['Cluter'] = y_predicted
coord['Latitude'] = ad['Latitude']
coord['Logtitude'] = ad['Logtitude']
coord['Cluter'] = ad['Cluter']
points = coord[['Latitude','Logtitude']].values.tolist()
point_center = km.cluster_centers_


In [ ]:
point_center

In [ ]:
cluter_ = []
for i in range(k):
    cluter_.append(coord[coord['Cluter'] == i])
    # cluter_.append(coord[['Latitude','Logtitude']].values.tolist())
cluter_


# color

In [100]:
# Tinh toa do trung binh
ave_lat = sum(p[0] for p in points)/len(points)
ave_log = sum(p[1] for p in points)/len(points)
file=folium.Map(location=[ave_lat,ave_log],zoom_start=15,min_zoom=1,max_zoom=20)
# coord['Latitude']

In [ ]:
for x in cluter_:
    fg = folium.FeatureGroup(name = 'Map')
    cl = random.choice(colors)
    print(cl)
    p = x[['Latitude','Logtitude']].values.tolist()
    # print(p)
    for i in p:
        fg.add_child(folium.Marker(location = [i[0],i[1]],popup=i[0],icon = folium.Icon(icon = "home", color = cl)))
    file.add_child(fg)
file
    
# cluter_

# CONVERT HULL

In [102]:
input_points = []
for x in cluter_:
    input_points.append(x[['Latitude','Logtitude']].values.tolist())

In [ ]:
input_points

In [104]:
# Python3 program to find convex hull of a set of points. Refer
# https://www.geeksforgeeks.org/orientation-3-ordered-points/
# for explanation of orientation()

# point class with x, y as point
class Point:
	def __init__(self, x, y):
		self.x = x
		self.y = y

def Left_index(points):
	
	'''
	Finding the left most point
	'''
	minn = 0
	for i in range(1,len(points)):
		if points[i].x < points[minn].x:
			minn = i
		elif points[i].x == points[minn].x:
			if points[i].y > points[minn].y:
				minn = i
	return minn

def orientation(p, q, r):
	'''
	To find orientation of ordered triplet (p, q, r).
	The function returns following values
	0 --> p, q and r are collinear
	1 --> Clockwise
	2 --> Counterclockwise
	'''
	val = (q.y - p.y) * (r.x - q.x) - \
		(q.x - p.x) * (r.y - q.y)

	if val == 0:
		return 0
	elif val > 0:
		return 1
	else:
		return 2

def convexHull(points, n):
	
	# There must be at least 3 points
	if n < 3:
		return

	# Find the leftmost point
	l = Left_index(points)

	hull = []
	
	'''
	Start from leftmost point, keep moving counterclockwise
	until reach the start point again. This loop runs O(h)
	times where h is number of points in result or output.
	'''
	p = l
	q = 0
	while(True):
		
		# Add current point to result
		hull.append(p)

		'''
		Search for a point 'q' such that orientation(p, q,
		x) is counterclockwise for all points 'x'. The idea
		is to keep track of last visited most counterclock-
		wise point in q. If any point 'i' is more counterclock-
		wise than q, then update q.
		'''
		q = (p + 1) % n

		for i in range(n):
			
			# If i is more counterclockwise
			# than current q, then update q
			if(orientation(points[p],
						points[i], points[q]) == 2):
				q = i

		'''
		Now q is the most counterclockwise with respect to p
		Set p as q for next iteration, so that q is added to
		result 'hull'
		'''
		p = q

		# While we don't come to first point
		if(p == l):
			break

	# Print Result
	# for each in hull:
	# 	print(points[each].x, points[each].y)
	ans = []

	for each in hull:
		ans.append([points[each].x, points[each].y])
	return ans

# Driver Code
list_convex_hull = []
for i in range(len(input_points)):
  list_convex_hull.append([])
for i in range(len(input_points)):
  points = []
  for point in input_points[i]:
    points.append(Point(point[0], point[1]))
  list_convex_hull[i] = convexHull(points, len(points))

# This code is contributed by
# Akarsh Somani, IIIT Kalyani


In [ ]:
list_convex_hull

# Vitual on map

In [106]:
for i in list_convex_hull:
    # for x in i:
        # print(x)
    if i == None:
        continue
    folium.PolyLine(i, color="green", weight=2.5, opacity=1).add_to(file)
    # print(len(i))
    # break

In [ ]:
dem = 0
for td in list_convex_hull:
  if td == None:
    continue
  td.append(td[0])
  path = folium.PolyLine(td,color=colors[dem],weight=2,popup='A line!',tooltip='First Line').add_to(file)
  dem+=1
# for x in point_center:
#     print(x)
    # file.add_child(folium.Marker(location=[x], popup="Toa do", icon=folium.Icon(icon = "home", color = "red")))  
point_center

In [108]:
for i in point_center:
    file.add_child(folium.Marker(location=[i[0],i[1]], popup="Toa do", icon=folium.Icon(icon = "fa-bus", color = "red", prefix='fa')))  

In [ ]:
file